In [ ]:
#notebook se kolje s ucenjem?...

import tensorflow as tf
import numpy as np
import skimage as ski
import skimage.data, skimage.transform
import models.vgg_16s as model
import importlib.util
from tqdm import trange
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
output_notebook()

from eval_helper import *
import libs.cylib as cylib

from datasets.cityscapes.cityscapes import CityscapesDataset
from datasets.cityscapes.cityscapes_info import class_info, class_color_map
import datasets.reader as reader

tf.app.flags.DEFINE_integer('img_width', 1024, '')
tf.app.flags.DEFINE_integer('img_height', 432, '')
tf.app.flags.DEFINE_integer('img_depth', 3, '')

tf.app.flags.DEFINE_string('dataset_dir',
    '/home/kivan/datasets/Cityscapes/tensorflow/records/1024x432/', '')
tf.app.flags.DEFINE_integer('max_steps', 10000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_integer('batch_size', 1, '')
tf.app.flags.DEFINE_integer('num_classes', 19, '')
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_boolean('is_training', False, '')

FLAGS = tf.app.flags.FLAGS

#g_net_dir = '/home/kivan/source/results/semseg/tf/nets/23_3_13:39:43/'
#g_net_dir = '/home/kivan/source/results/semseg/tf/nets/14_4_22-21-28/'
g_net_dir = '/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/'
g_model_path = g_net_dir + 'model.py'
spec = importlib.util.spec_from_file_location("model", g_model_path)
model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model)

Loading BokehJS ...

In [ ]:
def evaluate_model(sess, dataset, logits, labels):
  conf_mat = np.ascontiguousarray(np.zeros((FLAGS.num_classes, FLAGS.num_classes), dtype=np.uint64))
  for i in trange(dataset.num_examples()):
    #_, loss_value = sess.run([train_op, loss])
    out_logits, yt, q_size = sess.run([logits, labels, queue.size()])
    y = out_logits[0].argmax(2).astype(np.int32, copy=False)
    yt = yt.astype(np.int32, copy=False)
    cylib.collect_confusion_matrix(y.reshape(-1), yt, conf_mat)
    #print(q_size)
  return compute_errors(conf_mat, 'Validation', class_info, verbose=False)

p_iou = figure(plot_width=600, plot_height=400)
p_pix = figure(plot_width=600, plot_height=400)
p_iou.title = 'IoU accuracy'
p_pix.title = 'Pixel accuracy'
#p_iou.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)
# add a line renderer
#push_notebook()


pixel_acc_lst = []
iou_acc_lst = []
dataset = CityscapesDataset(FLAGS.dataset_dir, 'val')
ckpt = tf.train.get_checkpoint_state(g_net_dir)


if ckpt and ckpt.model_checkpoint_path:
  split_name = ckpt.model_checkpoint_path.split('/')[-1].split('-')
  prefix = split_name[-2]
  last_num = int(split_name[-1])
  for i in range(1, last_num+1):
    with tf.Graph().as_default():
      sess = tf.Session()
      image, labels, _, _, img_name, queue = reader.inputs_for_inference(sess, dataset)
      # Restores from checkpoint
      #with tf.Session() as sess:
      logits_sub = model.inference(image)
      logits = tf.image.resize_bilinear(logits_sub, [FLAGS.img_height, FLAGS.img_width])
      saver = tf.train.Saver()
      checkpoint_path = g_net_dir + prefix + '-' + str(i)
      print(checkpoint_path)
      saver.restore(sess, checkpoint_path)
      # Start the queue runners.
      coord = tf.train.Coordinator()
      data_threads = tf.train.start_queue_runners(sess=sess, coord=coord)
      pixel_acc, iou_acc, recall, precision, num_pixels = evaluate_model(sess, dataset, logits, labels)
      pixel_acc_lst += [pixel_acc]
      iou_acc_lst += [iou_acc]
      print(iou_acc)
      #tf.reset_default_graph() - bad
      #push_notebook()
      # must call before join
      coord.request_stop()
      sess.close()
      coord.join(data_threads)
  x = np.linspace(0, len(iou_acc_lst)-1, len(iou_acc_lst))
  p_iou.line(x, iou_acc_lst, line_width=2)
  p_pix.line(x, pixel_acc_lst, line_width=2)
  show(p_iou)
  show(p_pix)

else:
  print('No checkpoint file found')
  raise

/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-1


42.5225702353
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-2


48.9253136155
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-3


51.4307056396
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-4


53.5818570761
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-5


...

56.7664318238
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-11


56.6275262949
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-12


56.0128620383
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-13


56.151355025
/home/kivan/source/results/semseg/tf/nets/20_4_15-31-24/model.ckpt-14


 22%|██▏       | 11/50 [00:03<00:11,  3.44it/s]

[]